In [ ]:
# If running on Colab you will be asked to restart the session. This is normal and you should do so.
# Only run this cell if running on Colab.
# Change to the notebooks directory
%cd dsi-toolathon-2025-granite-geospatial-land-surface-temperature/notebooks/
%pwd

In [1]:
from rasterio.crs import CRS
import rioxarray as rxr
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.windows import Window
import rasterio.mask as riomask
import rasterio
from rasterio.crs import CRS
import glob
import os

In [2]:
def reproject_crs(filename, dst_crs= 'EPSG:4326'):
    """
    reprojects .tif file into given crs
    writes reprojected version to same file
    inputs:
        filename: .tif file to reproject
    """
    
    src = rasterio.open(filename)
    src_crs = str(src.crs)
    transform, width, height = calculate_default_transform(
                                        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
                        'crs': dst_crs,
                        'transform': transform,
                        'width': width,
                        'height': height
                    })
    #write reprojected version to save_to
    os.makedirs("../data/predictions-dsi/reprojected/", exist_ok=True)
    save_name = filename.replace("predictions-dsi", "predictions-dsi/reprojected/")
    with rasterio.open(save_name, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

In [3]:
files_to_reproject = glob.glob("../data/predictions-dsi/*.tif")
for file in files_to_reproject:
    reproject_crs(file)